In [2]:
############################# LOAD DATA
############################# LOad data, tf.data
import tensorflow as tf
train_dir='C:\\Users\\SANJAY\\SENTIMENT_ANALYSIS\\aclImdb\\train'
test_dir='C:\\Users\\SANJAY\\SENTIMENT_ANALYSIS\\aclImdb\\test'
batch_size=32
seed=123
raw_train_ds=tf.keras.preprocessing.text_dataset_from_directory(train_dir,batch_size=batch_size,
        seed=seed,validation_split=0.2,subset='training')
print(raw_train_ds.class_names)
raw_val_ds=tf.keras.preprocessing.text_dataset_from_directory(train_dir,batch_size=batch_size,
        seed=seed,validation_split=0.2,subset='validation')
raw_test_ds=tf.keras.preprocessing.text_dataset_from_directory(test_dir,batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
['neg', 'pos']
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [3]:
for text_batch,label_batch in raw_train_ds.take(1):
    for i in range(3):
        print('Review:-',text_batch[i].numpy())
        print('Label:-',label_batch[i].numpy())

Review:- b'After, I watched the films... I thought, "Why the heck was this film such a high success in the Korean Box Office?" Even thought the movie had a clever/unusal scenario, the acting wasn\'t that good and the characters weren\'t very interesting. For a Korean movie... I liked the fighting scenes. If you want to watch a film without thinking, this is the film for you. But I got to admit... the film was kind of childish... 6/10'
Label:- 1
Review:- b'Or released on DVD or screened on a cable channel like Amer. Life TV network. I have been watching another favorite, "Voyage to the Bottom of the Sea", as well as "Lost in Space" and Land of Giants". They\'ve been showing them forever but aren\'t receptive to suggestions for other shows. My father and I were big fans as I was already a big science/electronics nut, (still am) and my father was an old school chum of Nader. They both attended Oxy together. I still have memories of several of the episodes even though I was only 9. More so

In [4]:
print('Lebel 1',raw_train_ds.class_names[1])
print('Lebel 0',raw_train_ds.class_names[0])

Lebel 1 pos
Lebel 0 neg


In [5]:
import string
print(string.punctuation)
#.......

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [7]:
#######################preprocessing 
# s->A|b,cl
import re
def cust_stand(input_data):
    lower=tf.strings.lower(input_data)
    html=tf.strings.regex_replace(lower,'<br />','')
    punc=tf.strings.regex_replace(html,'[%s]'%re.escape(string.punctuation),'')
    return punc
max_features=10000 # dictionary size, number od words
seq_length=250 # fixed lenth
vectorize_layer=tf.keras.layers.experimental.preprocessing.TextVectorization(
    standardize=cust_stand,max_tokens=max_features,
                                 output_mode='int',output_sequence_length=seq_length)
train_text=raw_train_ds.map(lambda x,y:x)
vectorize_layer.adapt(train_text)


In [8]:
def vectorize_text(text,label):
    text=tf.expand_dims(text,-1) # batch
    return vectorize_layer(text),label

In [9]:
text_batch,label_batch=next(iter(raw_train_ds))
r1,l1=text_batch[0],label_batch[0]
print('Review-',r1)
print('Label',l1)
print('Vectorize text',vectorize_text(r1,l1))

Review- tf.Tensor(b'Poor Tobe Hopper. He directed an all time horror classic "Texas Chaimsaw Massacre". Since then everything he\'s done has been horrible. This is probably the worst...and that\'s saying a lot. It\'s about a man (Brad Dourif) who has the ability to make things (and people) catch fire...or something like that. Hardly an original idea (anyone remember "Firestarter"?) It\'s a real mess...literally EVERYTHING is done wrong! Pathetic acting (even Dourif!), asinine script, loust production values, crappy special effects...everything is BAD!!!!! A must miss...not even good for laughs.', shape=(), dtype=string)
Label tf.Tensor(0, shape=(), dtype=int32)
Vectorize text (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[ 339, 7903, 4642,   27,  531,   33,   31,   61,  196,  341, 2379,
           1, 3240,  233,   93,  288,  225,  219,   43,   73,  524,   10,
           7,  235,    2,    1,  178,  650,    4,  168,   29,   42,    4,
         130, 2853,    1,   35,   43,    2, 

In [10]:
print(vectorize_layer.get_vocabulary()[141])
print(vectorize_layer.get_vocabulary()[2870])

back
causes


In [11]:
train_ds=raw_train_ds.map(vectorize_text)
test_ds=raw_test_ds.map(vectorize_text)
val_ds=raw_val_ds.map(vectorize_text)

In [12]:
###################create model
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(10001,16)) # number of feature+1,embedding size
model.add(tf.keras.layers.Dropout(0.2)) # 20% random connection close
model.add(tf.keras.layers.GlobalAveragePooling1D()) # raplace a vector by aberage
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
hist=model.fit(train_ds,validation_data=val_ds,epochs=2)

Epoch 1/2
625/625 [==============================] - 59s 92ms/step - loss: 0.8332 - accuracy: 0.5278 - val_loss: 0.6789 - val_accuracy: 0.5696
Epoch 2/2
625/625 [==============================] - 13s 21ms/step - loss: 0.6318 - accuracy: 0.7122 - val_loss: 0.5914 - val_accuracy: 0.8184


In [14]:
test_loss,test_acc=model.evaluate(test_ds)
print(test_acc)

782/782 [==============================] - 230s 290ms/step - loss: 0.5978 - accuracy: 0.8000
0.7999600172042847


In [15]:
model_dict=hist.history
print(model_dict.keys())
print('Training acc',model_dict['accuracy'])

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
Training acc [0.5277500152587891, 0.7122499942779541]


In [17]:
export_model=tf.keras.Sequential([
    vectorize_layer,model,tf.keras.layers.Activation('sigmoid')
])
export_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
test_loss,test_acc=export_model.evaluate(raw_test_ds)
print(test_acc)

782/782 [==============================] - 11s 13ms/step - loss: 0.6951 - accuracy: 0.5000
0.5


In [19]:
import numpy as np
f=open('C:\\Users\\SANJAY\\SENTIMENT_ANALYSIS\\abc.txt','r')
text=f.read()
f.close()
print(text)
###############################
text_t=tf.constant(text)
text_t=tf.expand_dims(text_t,-1)
print(text_t)
#################
pred=export_model.predict(text_t)
print(pred)
out=np.where(pred<0.5,'NEG','POS')
print(out)

I like this movie.
tf.Tensor([b'I like this movie.'], shape=(1,), dtype=string)
[[0.63102347]]
[['POS']]
